In [1]:
from lsst.resources import ResourcePath
from astropy.io import fits
import os
import numpy
from astropy.table import Table
from dateutil.parser import parse
import re

#MC_C_20240918_000019_R40_SG1_guider.fits
prefix = 'MC_C_'
p = re.compile( rf'{prefix}(\d*)_(\d*)_(R.._S..)_guider.fits' )


In [2]:
def examineone(fname):
    date, seqnum, sensor = p.match(fname).groups()
    path = f's3://embargo@rubin-summit/LSSTCam/{date}/{prefix}{date}_{seqnum}/{fname}'
    rp=ResourcePath(path)
    with rp.open(mode="rb") as f:
        hdu_list = fits.open(f)
    stdnoise = numpy.mean(list(map( lambda x: numpy.std(x.data), hdu_list[1:]))[1:]) 
    stdstdnoise = numpy.std(list(map( lambda x: numpy.std(x.data), hdu_list[1:]))[1:]) 
    hz = 1/numpy.mean(numpy.diff(list(map( lambda x: parse(x.header['STMPTIME']), hdu_list[1:]) )[1:])).total_seconds()
    
    return {
        's3path': rp,
        'basename': rp.abspath().basename(),
        'noise': stdnoise,
        'std(noise)': stdstdnoise,        
        'Hz': hz
    }
    

In [3]:
Table(
    list(map( lambda x: examineone(x), [
        'MC_C_20240918_000019_R40_SG1_guider.fits',
        'MC_C_20240916_000327_R00_SG0_guider.fits'
    ])
       )
)['basename','noise','std(noise)','Hz']

basename,noise,std(noise),Hz
str40,float64,float64,float64
MC_C_20240918_000019_R40_SG1_guider.fits,9.212806203560172,0.12484796887863193,9.163131225202276
MC_C_20240916_000327_R00_SG0_guider.fits,4.6481357175004225,0.01161241270003768,2.2182490916269972
